# Qaintellect training example 2: multiple qubits and measurement operators

In this example, we will demonstrate the optimisation of a quantum circuit with multiple qubits and measurement operators.

In [1]:
include("../src/Qaintellect.jl")
using .Qaintellect

using LinearAlgebra
using Flux
using IterTools: ncycle

We create a parametrized quantum circuit with $N$ qubits. We then optimize these parameters using the Flux framework.

In [2]:
# construct parametrized circuit

N = 4

ry = RyGate(√2)
n = randn(Float64, 3)
n /= norm(n)
rg = RotationGate(0.2π, n)
@show(rg)

cgc = [
    circuit_gate(3, HadamardGate()),
    circuit_gate(2, RzGate(1.5π), (1, 4)), # controlled gate
    circuit_gate(2, 3, SwapGate()),
    circuit_gate(3, PhaseShiftGate(0.3)),
    circuit_gate(3, rg),
    circuit_gate(1, ry),
]
meas = [MeasurementOperator(Matrix{Float64}(I, 2^N, 2^N), Tuple(1:N)), MeasurementOperator(Hermitian(randn(ComplexF64, 2^N, 2^N)), Tuple(1:N))]

c = Circuit{N}(cgc, meas)

rg = RotationGate([-0.18283223885930364, -0.12268599911365217, -0.5884765875536203])



    4 ————————•—————————————————————
              |                     
    3 —[H ]—————————x————[Pϕ]——[Rθ]—
              |     |               
    2 ———————[Rz]———x———————————————
              |                     
    1 ————————•————————————————[Ry]—


In [3]:
# set up model
model(ψ) = dot([0.3, -1.2], c(ψ))

# create loss function
loss(x, y) = Flux.mse(model(x), y)

# gather parameters from Circuit
paras = Flux.params(c)

# freeze parameter ry.θ, equivalent to delete!(paras, Qaintessent.get_trainable(ry))
delete!(paras, ry.θ)

paras

Params([[4.71238898038469], [0.3], [-0.18283223885930364, -0.12268599911365217, -0.5884765875536203], 
  [1 ,  1]  =  1.0+0.0im
  [2 ,  2]  =  1.0+0.0im
  [3 ,  3]  =  1.0+0.0im
  [4 ,  4]  =  1.0+0.0im
  [5 ,  5]  =  1.0+0.0im
  [6 ,  6]  =  1.0+0.0im
  [7 ,  7]  =  1.0+0.0im
  [8 ,  8]  =  1.0+0.0im
  [9 ,  9]  =  1.0+0.0im
  [10, 10]  =  1.0+0.0im
  [11, 11]  =  1.0+0.0im
  [12, 12]  =  1.0+0.0im
  [13, 13]  =  1.0+0.0im
  [14, 14]  =  1.0+0.0im
  [15, 15]  =  1.0+0.0im
  [16, 16]  =  1.0+0.0im, 
  [1 ,  1]  =  0.2806+0.0im
  [2 ,  1]  =  -0.361313-0.0955465im
  [3 ,  1]  =  0.591408-1.39305im
  [4 ,  1]  =  0.245174+1.03142im
  [5 ,  1]  =  -0.773017+0.0652003im
  [6 ,  1]  =  0.546163+0.338735im
  [7 ,  1]  =  0.45597-0.940909im
  [8 ,  1]  =  -0.327946-1.01356im
  [9 ,  1]  =  0.57369+0.697424im
  [10,  1]  =  0.93837+0.605664im
  [11,  1]  =  0.632955-1.26973im
  [12,  1]  =  -0.760811+0.808969im
  ⋮
  [4 , 16]  =  0.148326+0.436451im
  [5 , 16]  =  -1.15716-0.280364im
  [6 , 16

In [4]:
# input quantum state
ψ = randn(ComplexF64, 2^N)
ψ /= norm(ψ)

# desired output
e = 0.65

# set up data for training
data = ncycle([(ψ, e)], 64)

# define optimizer
opt = Descent(1)

# define evaluation function
evalcb() = @show(loss(ψ, e))

println("Initial model evaluation: $(model(ψ)), target: $e")

Flux.train!(loss, paras, data, opt, cb=Flux.throttle(evalcb, 0.01))

println("Final model evaluation: $(model(ψ)), target: $e")

Initial model evaluation: 1.700167730760974, target: 0.65
loss(ψ, e) = 1.2740055736258067
loss(ψ, e) = 0.005736894262053143
loss(ψ, e) = 0.00019023618803027756
loss(ψ, e) = 7.486929136287459e-9
loss(ψ, e) = 8.262051740172539e-14
loss(ψ, e) = 5.174603429777902e-16
loss(ψ, e) = 1.6063392435573423e-21
loss(ψ, e) = 4.9857981362231e-27
loss(ψ, e) = 2.8398992587956425e-29
Final model evaluation: 0.6500000000000001, target: 0.65


In [5]:
# check: Ry gate has not changed (frozen parameter)
@show(ry)

# on the other hand, parameters of general rotation gate are now different
@show(rg);

ry = RyGate([1.4142135623730951])
rg = RotationGate([6.378625988509723, 2.558620861831795, -1.1728874898408725])
